In [1]:
!pip install fastbook
import fastbook
# After installing fastbook, we will install fastai vision

from fastbook import *
from fastai.vision.widgets import *

In [3]:
path = '../input/weather-dataset/dataset'
hog = get_image_files(path)
hog

In [6]:
weathers = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label
)

In [7]:
weather = weathers.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
)

In [8]:
dls = weathers.dataloaders(path)

In [9]:
dls.valid.show_batch(max_n=12, nrows=4) 

In [11]:
learn = cnn_learner(dls, resnet50, metrics=error_rate)
learn.fine_tune(epochs=10)

In [13]:
from IPython.core.pylabtools import figsize 
interp = ClassificationInterpretation.from_learner(learn) 
interp.plot_confusion_matrix(figsize=(10,10))

In [14]:
interp.plot_top_losses(10, nrows=2, figsize=(20,6))

In [15]:
learn.export(fname='resnet50.pkl')

In [16]:
!pip install streamlit

In [17]:
import pathlib
from pathlib import Path
import streamlit as st
from fastai.vision.all import *
from fastai.vision.widgets import *

In [18]:
learn_inf = load_learner('resnet50.pkl')


In [36]:
uploader = widgets.FileUpload()
uploader

In [41]:
img = PILImage.create(uploader.data[0])
sxs,_,probs = learn.predict(img)
print(f"What is this --->>:: {sxs}")
print(f"Probability peaks at :: {probs[1].item()*100}")